Student ID : 20670423

In [1]:
#Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("train.csv")

In [3]:
#Data pre-processing

df['is_free'] =df['is_free'].apply(lambda x: 1 if x==True else 0)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['release_date'] = pd.to_datetime(df['release_date'])
df['Difference'] = df['purchase_date'].sub(df['release_date'], axis=0)
df['Difference'] = df['Difference'] / np.timedelta64(1, 'D')
df['Difference']=df['Difference'].fillna(df['Difference'].mean())
df['total_positive_reviews']=df['total_positive_reviews'].fillna(df['total_positive_reviews'].mean())
df['total_negative_reviews']=df['total_negative_reviews'].fillna(df['total_negative_reviews'].mean())

In [4]:
dff = df.drop(columns=['purchase_date','release_date'])
dff

,id,playtime_forever,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference
0,0,0.000000,0,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",372.0,96.0,1665.0
1,1,0.016667,1,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",23.0,0.0,472.0
2,2,0.000000,0,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",3018.0,663.0,1616.0
3,3,1.533333,0,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",63078.0,1746.0,2434.0
4,4,22.333333,0,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",8841.0,523.0,2043.0
...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",150.0,91.0,339.0
353,353,0.000000,0,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",19008.0,4849.0,1149.0
354,354,0.000000,0,8300.0,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",5099.0,1719.0,817.0
355,355,0.000000,0,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",718.0,159.0,477.0


In [5]:
#Tokenizing the columns genres, categories and tags

column = ['genres','categories','tags']
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,' ')
    return text
for i in column:
    dff.loc[:,i]=df.loc[:,i].apply(remove_punctuations)
dff

,id,playtime_forever,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference
0,0,0.000000,0,3700.0,Adventure Casual Indie,Single player Steam Trading Cards Steam Cloud,Indie Adventure Story Rich Casual Atmospheric ...,372.0,96.0,1665.0
1,1,0.016667,1,0.0,RPG,Single player Partial Controller Support,Mod Utilities RPG Game Development Singleplaye...,23.0,0.0,472.0
2,2,0.000000,0,5000.0,Adventure Casual Indie,Single player Full controller support Steam Tr...,Point Click Adventure Story Rich Comedy Indi...,3018.0,663.0,1616.0
3,3,1.533333,0,9900.0,Action RPG,Single player Multi player Steam Achievements ...,Medieval RPG Open World Strategy Sandbox Actio...,63078.0,1746.0,2434.0
4,4,22.333333,0,4800.0,Action Indie Strategy,Single player Co op Steam Achievements Full co...,Tower Defense Co op Action Strategy Online Co ...,8841.0,523.0,2043.0
...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,Action Simulation,Single player Multi player Online Multi Player...,Simulation Action VR Flight,150.0,91.0,339.0
353,353,0.000000,0,6800.0,Action Adventure,Single player Steam Achievements Full controll...,Batman Action Open World Superhero Stealth Adv...,19008.0,4849.0,1149.0
354,354,0.000000,0,8300.0,Action Adventure RPG,Single player Steam Achievements Full controll...,Action Hack and Slash Adventure RPG Open World...,5099.0,1719.0,817.0
355,355,0.000000,0,6800.0,Action Adventure,Single player Steam Achievements Full controll...,Action FPS Adventure Shooter Cyberpunk Robots ...,718.0,159.0,477.0


In [6]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer


In [7]:


vect = CountVectorizer()
g = vect.fit_transform(dff['genres'])
vect.get_feature_names()
a = pd.DataFrame(g.toarray(), columns = vect.get_feature_names())

vect = CountVectorizer()
c = vect.fit_transform(dff['categories'])
vect.get_feature_names()
b = pd.DataFrame(c.toarray(), columns = vect.get_feature_names())


vect = CountVectorizer()
t = vect.fit_transform(dff['tags'])
vect.get_feature_names()
d = pd.DataFrame(t.toarray(), columns = vect.get_feature_names())






In [8]:

df_new = pd.concat([dff,a,b,d], axis=1)
df_new

,id,playtime_forever,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference,...,war,wargame,warhammer,wars,western,with,workshop,world,your,zombies
0,0,0.000000,0,3700.0,Adventure Casual Indie,Single player Steam Trading Cards Steam Cloud,Indie Adventure Story Rich Casual Atmospheric ...,372.0,96.0,1665.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.016667,1,0.0,RPG,Single player Partial Controller Support,Mod Utilities RPG Game Development Singleplaye...,23.0,0.0,472.0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.000000,0,5000.0,Adventure Casual Indie,Single player Full controller support Steam Tr...,Point Click Adventure Story Rich Comedy Indi...,3018.0,663.0,1616.0,...,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,0,9900.0,Action RPG,Single player Multi player Steam Achievements ...,Medieval RPG Open World Strategy Sandbox Actio...,63078.0,1746.0,2434.0,...,0,0,0,0,0,0,0,1,0,0
4,4,22.333333,0,4800.0,Action Indie Strategy,Single player Co op Steam Achievements Full co...,Tower Defense Co op Action Strategy Online Co ...,8841.0,523.0,2043.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,0.783333,0,8800.0,Action Simulation,Single player Multi player Online Multi Player...,Simulation Action VR Flight,150.0,91.0,339.0,...,0,0,0,0,0,0,0,0,0,0
353,353,0.000000,0,6800.0,Action Adventure,Single player Steam Achievements Full controll...,Batman Action Open World Superhero Stealth Adv...,19008.0,4849.0,1149.0,...,0,0,0,0,0,0,0,1,0,0
354,354,0.000000,0,8300.0,Action Adventure RPG,Single player Steam Achievements Full controll...,Action Hack and Slash Adventure RPG Open World...,5099.0,1719.0,817.0,...,0,0,0,0,0,0,0,1,0,0
355,355,0.000000,0,6800.0,Action Adventure,Single player Steam Achievements Full controll...,Action FPS Adventure Shooter Cyberpunk Robots ...,718.0,159.0,477.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
d3 = df_new.drop(columns=['genres','categories','tags','id'])

In [10]:
d3

,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,animation,...,war,wargame,warhammer,wars,western,with,workshop,world,your,zombies
0,0.000000,0,3700.0,372.0,96.0,1665.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,1,0.0,23.0,0.0,472.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0,5000.0,3018.0,663.0,1616.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,0,9900.0,63078.0,1746.0,2434.0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,22.333333,0,4800.0,8841.0,523.0,2043.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0.783333,0,8800.0,150.0,91.0,339.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
353,0.000000,0,6800.0,19008.0,4849.0,1149.0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
354,0.000000,0,8300.0,5099.0,1719.0,817.0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
355,0.000000,0,6800.0,718.0,159.0,477.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
d3.iloc[:, 1:442]


,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,animation,audio,...,war,wargame,warhammer,wars,western,with,workshop,world,your,zombies
0,0,3700.0,372.0,96.0,1665.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,23.0,0.0,472.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5000.0,3018.0,663.0,1616.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,9900.0,63078.0,1746.0,2434.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,4800.0,8841.0,523.0,2043.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0,8800.0,150.0,91.0,339.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353,0,6800.0,19008.0,4849.0,1149.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
354,0,8300.0,5099.0,1719.0,817.0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
355,0,6800.0,718.0,159.0,477.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Reading test dataset
dft = pd.read_csv("test.csv")
dft

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...","Oct 28, 2018",22-May-12,2607.0,1122.0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Jul 20, 2019",24-Apr-18,5762.0,2235.0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...","Jul 16, 2019",20-Mar-17,687.0,133.0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical","Jan 30, 2018",17-Jul-07,67.0,39.0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...","Feb 24, 2017",22-Aug-13,40344.0,3708.0
...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...","Mar 23, 2018",22-May-13,9334.0,516.0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...","Mar 3, 2018",14-Sep-09,1245.0,176.0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...","Feb 18, 2018",13-Aug-13,33128.0,6350.0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...","Nov 19, 2017",19-Jun-17,5.0,0.0


In [13]:
#Doing some basic pre=processing of test dataset
dft['is_free'] =dft['is_free'].apply(lambda x: 1 if x==True else 0)
dft['purchase_date'] = pd.to_datetime(dft['purchase_date'])
dft['release_date'] = pd.to_datetime(dft['release_date'])
dft['Difference'] = dft['purchase_date'].sub(dft['release_date'], axis=0)
dft['Difference'] = dft['Difference'] / np.timedelta64(1, 'D')
dft['Difference']=dft['Difference'].fillna(dft['Difference'].mean())
dft['total_positive_reviews']=dft['total_positive_reviews'].fillna(dft['total_positive_reviews'].mean())
dft['total_negative_reviews']=dft['total_negative_reviews'].fillna(dft['total_negative_reviews'].mean())

In [14]:
dftt = dft.drop(columns=['purchase_date','release_date'])
dftt

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference
0,0,0,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2607.0,1122.0,2350.0
1,1,0,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",5762.0,2235.0,452.0
2,2,0,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",687.0,133.0,848.0
3,3,0,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",67.0,39.0,3850.0
4,4,0,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",40344.0,3708.0,1282.0
...,...,...,...,...,...,...,...,...,...
85,85,0,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",9334.0,516.0,1766.0
86,86,0,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",1245.0,176.0,3092.0
87,87,0,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",33128.0,6350.0,1650.0
88,88,1,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",5.0,0.0,153.0


In [15]:
colss = ['genres','categories','tags']
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,' ')
    return text
for i in colss:
    dftt.loc[:,i]=dftt.loc[:,i].apply(remove_punctuations)
dftt

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference
0,0,0,3500,Action Adventure,Single player Full controller support,Action Adventure Horror Third Person Singlepla...,2607.0,1122.0,2350.0
1,1,0,11600,Action Adventure Strategy,Single player Multi player Online Multi Player...,Mechs Strategy Turn Based Turn Based Tactics S...,5762.0,2235.0,452.0
2,2,0,2100,Indie Simulation Strategy,Single player Steam Achievements Steam Trading...,Strategy Simulation Indie Political Cold War P...,687.0,133.0,848.0
3,3,0,3600,Action Strategy,Single player Multi player Co op,Strategy Action Military Tactical,67.0,39.0,3850.0
4,4,0,3400,Action Adventure,Single player Co op Steam Achievements Full co...,Open World Action Comedy Co op Third Person Sh...,40344.0,3708.0,1282.0
...,...,...,...,...,...,...,...,...,...
85,85,0,5000,Action,Single player Steam Achievements Full controll...,Western FPS Action Story Rich Bullet Time Shoo...,9334.0,516.0,1766.0
86,86,0,3600,Action Adventure,Single player,Adventure Action Ninja Stealth Cute Singleplay...,1245.0,176.0,3092.0
87,87,0,11200,Simulation Strategy,Single player Multi player Cross Platform Mult...,Grand Strategy Strategy Historical Simulation ...,33128.0,6350.0,1650.0
88,88,1,0,Action Adventure RPG Simulation Sports Strategy,Single player Multi player Co op Online Co op ...,VR RPG Action Sports Simulation Adventure Stra...,5.0,0.0,153.0


In [16]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
gt = vect.fit_transform(dftt['genres'])
vect.get_feature_names()
z = pd.DataFrame(gt.toarray(), columns = vect.get_feature_names())

vect = CountVectorizer()
ct = vect.fit_transform(dftt['categories'])
vect.get_feature_names()
u = pd.DataFrame(ct.toarray(), columns = vect.get_feature_names())


vect = CountVectorizer()
tt = vect.fit_transform(dftt['tags'])
vect.get_feature_names()
k = pd.DataFrame(tt.toarray(), columns = vect.get_feature_names())

In [17]:
dft_new = pd.concat([dftt,z,u,k], axis=1)
dft_new

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,Difference,access,...,visual,vr,walking,war,warhammer,western,with,workshop,world,zombies
0,0,0,3500,Action Adventure,Single player Full controller support,Action Adventure Horror Third Person Singlepla...,2607.0,1122.0,2350.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,11600,Action Adventure Strategy,Single player Multi player Online Multi Player...,Mechs Strategy Turn Based Turn Based Tactics S...,5762.0,2235.0,452.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2100,Indie Simulation Strategy,Single player Steam Achievements Steam Trading...,Strategy Simulation Indie Political Cold War P...,687.0,133.0,848.0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,3600,Action Strategy,Single player Multi player Co op,Strategy Action Military Tactical,67.0,39.0,3850.0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,3400,Action Adventure,Single player Co op Steam Achievements Full co...,Open World Action Comedy Co op Third Person Sh...,40344.0,3708.0,1282.0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,0,5000,Action,Single player Steam Achievements Full controll...,Western FPS Action Story Rich Bullet Time Shoo...,9334.0,516.0,1766.0,0,...,0,0,0,0,0,1,0,0,0,0
86,86,0,3600,Action Adventure,Single player,Adventure Action Ninja Stealth Cute Singleplay...,1245.0,176.0,3092.0,0,...,0,0,0,0,0,0,0,0,1,0
87,87,0,11200,Simulation Strategy,Single player Multi player Cross Platform Mult...,Grand Strategy Strategy Historical Simulation ...,33128.0,6350.0,1650.0,0,...,0,0,0,0,0,0,1,0,1,0
88,88,1,0,Action Adventure RPG Simulation Sports Strategy,Single player Multi player Co op Online Co op ...,VR RPG Action Sports Simulation Adventure Stra...,5.0,0.0,153.0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
d6 = dft_new.drop(columns=['genres','categories','tags','id'])

In [19]:
d6

,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,casual,early,...,visual,vr,walking,war,warhammer,western,with,workshop,world,zombies
0,0,3500,2607.0,1122.0,2350.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,11600,5762.0,2235.0,452.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2100,687.0,133.0,848.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,3600,67.0,39.0,3850.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,3400,40344.0,3708.0,1282.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,5000,9334.0,516.0,1766.0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
86,0,3600,1245.0,176.0,3092.0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
87,0,11200,33128.0,6350.0,1650.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
88,1,0,5.0,0.0,153.0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [20]:
list=[]
for i in d3:
    if i not in d6:
        list.append(i)
print(list)
len(list)

['playtime_forever', 'animation', 'audio', 'design', 'illustration', 'modeling', 'production', 'racing', 'utilities', 'anti', 'cheat', 'enabled', 'valve', 'addictive', 'animation', 'artificial', 'arts', 'attack', 'atv', 'audio', 'automation', 'awkward', 'batman', 'battle', 'bikes', 'board', 'book', 'campaign', 'capitalism', 'card', 'cartoon', 'cats', 'chess', 'choose', 'clicker', 'comic', 'conspiracy', 'controls', 'croft', 'death', 'defense', 'design', 'development', 'documentary', 'dungeons', 'endings', 'events', 'experience', 'fighter', 'flight', 'game', 'god', 'gothic', 'gun', 'hell', 'hidden', 'horses', 'illustration', 'immersive', 'intelligence', 'intentionally', 'investigation', 'lara', 'lgbtq', 'logic', 'mars', 'martial', 'metroidvania', 'mmorpg', 'modeling', 'motocross', 'motorbike', 'movie', 'multiple', 'naval', 'object', 'offroad', 'own', 'perma', 'pirates', 'production', 'programming', 'pve', 'quick', 'racing', 'rome', 'royale', 'sailing', 'satire', 'score', 'sequel', 'snipe

116

In [21]:
[item for item in d3 if item not in d6]



['playtime_forever',
 'animation',
 'audio',
 'design',
 'illustration',
 'modeling',
 'production',
 'racing',
 'utilities',
 'anti',
 'cheat',
 'enabled',
 'valve',
 'addictive',
 'animation',
 'artificial',
 'arts',
 'attack',
 'atv',
 'audio',
 'automation',
 'awkward',
 'batman',
 'battle',
 'bikes',
 'board',
 'book',
 'campaign',
 'capitalism',
 'card',
 'cartoon',
 'cats',
 'chess',
 'choose',
 'clicker',
 'comic',
 'conspiracy',
 'controls',
 'croft',
 'death',
 'defense',
 'design',
 'development',
 'documentary',
 'dungeons',
 'endings',
 'events',
 'experience',
 'fighter',
 'flight',
 'game',
 'god',
 'gothic',
 'gun',
 'hell',
 'hidden',
 'horses',
 'illustration',
 'immersive',
 'intelligence',
 'intentionally',
 'investigation',
 'lara',
 'lgbtq',
 'logic',
 'mars',
 'martial',
 'metroidvania',
 'mmorpg',
 'modeling',
 'motocross',
 'motorbike',
 'movie',
 'multiple',
 'naval',
 'object',
 'offroad',
 'own',
 'perma',
 'pirates',
 'production',
 'programming',
 'pve',
 

In [22]:
[item for item in d6 if item not in d3]

['cold', 'gamemaker', 'lore', 'modern', 'rails']

In [23]:
list.remove('playtime_forever')

In [24]:
list

['animation',
 'audio',
 'design',
 'illustration',
 'modeling',
 'production',
 'racing',
 'utilities',
 'anti',
 'cheat',
 'enabled',
 'valve',
 'addictive',
 'animation',
 'artificial',
 'arts',
 'attack',
 'atv',
 'audio',
 'automation',
 'awkward',
 'batman',
 'battle',
 'bikes',
 'board',
 'book',
 'campaign',
 'capitalism',
 'card',
 'cartoon',
 'cats',
 'chess',
 'choose',
 'clicker',
 'comic',
 'conspiracy',
 'controls',
 'croft',
 'death',
 'defense',
 'design',
 'development',
 'documentary',
 'dungeons',
 'endings',
 'events',
 'experience',
 'fighter',
 'flight',
 'game',
 'god',
 'gothic',
 'gun',
 'hell',
 'hidden',
 'horses',
 'illustration',
 'immersive',
 'intelligence',
 'intentionally',
 'investigation',
 'lara',
 'lgbtq',
 'logic',
 'mars',
 'martial',
 'metroidvania',
 'mmorpg',
 'modeling',
 'motocross',
 'motorbike',
 'movie',
 'multiple',
 'naval',
 'object',
 'offroad',
 'own',
 'perma',
 'pirates',
 'production',
 'programming',
 'pve',
 'quick',
 'racing',
 

In [25]:
f = np.zeros(shape=(90,115))
dummy = pd.DataFrame(f,columns=list)
dummy.astype(int)

,animation,audio,design,illustration,modeling,production,racing,utilities,anti,cheat,...,transportation,twin,typing,underground,unforgiving,utilities,voxel,wargame,wars,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
dft_1 = pd.concat([d6,dummy], axis=1)
dft_1

,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,casual,early,...,transportation,twin,typing,underground,unforgiving,utilities,voxel,wargame,wars,your
0,0,3500,2607.0,1122.0,2350.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,11600,5762.0,2235.0,452.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2100,687.0,133.0,848.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3600,67.0,39.0,3850.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3400,40344.0,3708.0,1282.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,5000,9334.0,516.0,1766.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0,3600,1245.0,176.0,3092.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0,11200,33128.0,6350.0,1650.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1,0,5.0,0.0,153.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
[item for item in dft_1 if item not in d3]

['cold', 'gamemaker', 'lore', 'modern', 'rails']

In [28]:
d6_new = dft_1.drop(columns=['cold', 'gamemaker', 'lore', 'modern', 'rails'])
d6_new

,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,casual,early,...,transportation,twin,typing,underground,unforgiving,utilities,voxel,wargame,wars,your
0,0,3500,2607.0,1122.0,2350.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,11600,5762.0,2235.0,452.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2100,687.0,133.0,848.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3600,67.0,39.0,3850.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3400,40344.0,3708.0,1282.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,5000,9334.0,516.0,1766.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0,3600,1245.0,176.0,3092.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0,11200,33128.0,6350.0,1650.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1,0,5.0,0.0,153.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
[item for item in d6_new if item not in d3]

[]

In [30]:
d3

,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,animation,...,war,wargame,warhammer,wars,western,with,workshop,world,your,zombies
0,0.000000,0,3700.0,372.0,96.0,1665.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,1,0.0,23.0,0.0,472.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0,5000.0,3018.0,663.0,1616.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,0,9900.0,63078.0,1746.0,2434.0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,22.333333,0,4800.0,8841.0,523.0,2043.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,0.783333,0,8800.0,150.0,91.0,339.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
353,0.000000,0,6800.0,19008.0,4849.0,1149.0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
354,0.000000,0,8300.0,5099.0,1719.0,817.0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
355,0.000000,0,6800.0,718.0,159.0,477.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#Test after adding 116 and dropping 5
d6_new

,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,casual,early,...,transportation,twin,typing,underground,unforgiving,utilities,voxel,wargame,wars,your
0,0,3500,2607.0,1122.0,2350.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,11600,5762.0,2235.0,452.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2100,687.0,133.0,848.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3600,67.0,39.0,3850.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3400,40344.0,3708.0,1282.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,5000,9334.0,516.0,1766.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0,3600,1245.0,176.0,3092.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0,11200,33128.0,6350.0,1650.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1,0,5.0,0.0,153.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
d6_new.equals(d3)

False

In [33]:
frodo = sorted(d6_new)
sam = sorted(d3)

In [34]:
frodo

['1980s',
 '1990',
 '2d',
 '3d',
 '40k',
 '4x',
 '5d',
 'Difference',
 'access',
 'access',
 'achievements',
 'action',
 'action',
 'addictive',
 'adventure',
 'adventure',
 'aliens',
 'alternate',
 'america',
 'and',
 'animation',
 'animation',
 'anime',
 'anti',
 'apocalyptic',
 'app',
 'arcade',
 'arena',
 'artificial',
 'arts',
 'assassin',
 'atmospheric',
 'attack',
 'atv',
 'audio',
 'audio',
 'automation',
 'available',
 'awkward',
 'base',
 'based',
 'batman',
 'battle',
 'beat',
 'beautiful',
 'benchmark',
 'bikes',
 'blood',
 'board',
 'book',
 'builder',
 'building',
 'bullet',
 'campaign',
 'capitalism',
 'captions',
 'card',
 'cards',
 'cartoon',
 'cartoony',
 'casual',
 'casual',
 'cats',
 'character',
 'cheat',
 'chess',
 'choices',
 'choose',
 'cinematic',
 'city',
 'classic',
 'click',
 'clicker',
 'cloud',
 'co',
 'co',
 'collectibles',
 'colorful',
 'combat',
 'comedy',
 'comic',
 'commentary',
 'competitive',
 'conspiracy',
 'content',
 'controller',
 'controller',


In [35]:
sam

['1980s',
 '1990',
 '2d',
 '3d',
 '40k',
 '4x',
 '5d',
 'Difference',
 'access',
 'access',
 'achievements',
 'action',
 'action',
 'addictive',
 'adventure',
 'adventure',
 'aliens',
 'alternate',
 'america',
 'and',
 'animation',
 'animation',
 'anime',
 'anti',
 'apocalyptic',
 'app',
 'arcade',
 'arena',
 'artificial',
 'arts',
 'assassin',
 'atmospheric',
 'attack',
 'atv',
 'audio',
 'audio',
 'automation',
 'available',
 'awkward',
 'base',
 'based',
 'batman',
 'battle',
 'beat',
 'beautiful',
 'benchmark',
 'bikes',
 'blood',
 'board',
 'book',
 'builder',
 'building',
 'bullet',
 'campaign',
 'capitalism',
 'captions',
 'card',
 'cards',
 'cartoon',
 'cartoony',
 'casual',
 'casual',
 'cats',
 'character',
 'cheat',
 'chess',
 'choices',
 'choose',
 'cinematic',
 'city',
 'classic',
 'click',
 'clicker',
 'cloud',
 'co',
 'co',
 'collectibles',
 'colorful',
 'combat',
 'comedy',
 'comic',
 'commentary',
 'competitive',
 'conspiracy',
 'content',
 'content',
 'controller',
 'c

In [210]:
#set(sam) ^ set(frodo)


In [37]:
p=[]
frodo=d3.columns.tolist()
sam=d6_new.columns.tolist()
for i in d3:
    if frodo.count(i)>sam.count(i) and i not in p:
        p.append(i)
p

['playtime_forever', 'content', 'sexual']

In [38]:
ff = np.zeros(shape=(90,2))
dummyy = pd.DataFrame(ff,columns=['content','sexual'])
dummyy.astype(int)

,content,sexual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
85,0,0
86,0,0
87,0,0
88,0,0


In [39]:
d6_new2 = pd.concat([d6_new,dummyy], axis=1)
d6_new2

,is_free,price,total_positive_reviews,total_negative_reviews,Difference,access,action,adventure,casual,early,...,typing,underground,unforgiving,utilities,voxel,wargame,wars,your,content,sexual
0,0,3500,2607.0,1122.0,2350.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,11600,5762.0,2235.0,452.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2100,687.0,133.0,848.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3600,67.0,39.0,3850.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3400,40344.0,3708.0,1282.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0,5000,9334.0,516.0,1766.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0,3600,1245.0,176.0,3092.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0,11200,33128.0,6350.0,1650.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1,0,5.0,0.0,153.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as sfs



#model=LinearRegression()
#clf=PassiveAggressiveRegressor(random_state=0)
#k = KNeighborsRegressor()
#Decisiont = DecisionTreeRegressor()
#Radius = RadiusNeighborsRegressor() doesn't work
bagging = BaggingRegressor()
#Aboost = AdaBoostRegressor()

X_train=d3.iloc[:,1:442]
y_train=d3['playtime_forever']

bagging.fit(X_train, y_train)
#print(model.coef_)
#print(model.intercept_)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

In [49]:
X_test=d6_new2.iloc[:,0:441]
y_test=bagging.predict(X_test)
y_test

array([1.07833333e+00, 7.28333333e-01, 5.70000000e-01, 2.30000000e-01,
       1.53500000e+00, 3.15000000e-01, 1.38666667e+00, 6.18333333e-01,
       1.30166667e+00, 1.01500000e+00, 7.28333333e-01, 7.95000000e-01,
       1.92166667e+00, 2.95333333e+00, 3.45000000e+00, 1.51333333e+00,
       3.56333333e+00, 3.23166667e+00, 4.68333333e-01, 3.45500000e+00,
       1.97666667e+00, 2.24833333e+00, 2.07333333e+00, 7.51666667e-01,
       8.06666667e-01, 2.74833333e+00, 3.18833333e+00, 2.02833333e+00,
       3.85333333e+00, 1.67166667e+00, 1.34500000e+00, 2.47050000e+01,
       1.51833333e+00, 1.16333333e+00, 3.25833333e+00, 6.53333333e-01,
       4.19500000e+00, 1.53833333e+00, 1.94666667e+00, 1.79666667e+00,
       3.71833333e+00, 3.26666667e+00, 2.23666667e+00, 1.13666667e+00,
       3.27000000e+00, 3.52166667e+00, 4.01500000e+00, 9.75000000e-01,
       3.69666667e+00, 1.70516667e+01, 3.51333333e+00, 3.41833333e+00,
       3.09833333e+00, 5.23333333e+00, 3.49166667e+00, 1.34333333e+00,
      

In [50]:
y_test[y_test<0]=0

In [51]:
finale=pd.DataFrame({'id':np.arange(90),'playtime_forever':y_test})
finale

,id,playtime_forever
0,0,1.078333
1,1,0.728333
2,2,0.570000
3,3,0.230000
4,4,1.535000
...,...,...
85,85,5.323333
86,86,0.553333
87,87,1.943333
88,88,0.023333


In [53]:
finale.to_csv('finale(thisonemaybe)).csv',index=False)